In [1]:
import os
import json
import numpy as np
from PIL import Image
from torch.utils import data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.autograd import Variable
import torchvision.transforms as transforms
import time

from autoencoder import autoencoder, train, evaluate
from dataset import Dataset

# Model

In [2]:
save_model_path = './ae_models/ae/'

# EncoderCNN architecture
fc_hidden1 = 256
embed_dim = 32     # latent dim extracted by 2D CNN
dropout_p = 0.2       # dropout probability

# training parameters
start_epoch = 14
epochs = 20     # training epochs
batch_size = 8
learning_rate = 1e-3
log_interval = 1000   # interval for displaying training info

In [4]:
# Detect devices
use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu")   # use CPU or GPU

# Data loading parameters
params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 4, 'pin_memory': True} if use_cuda else {}
transform = transforms.Compose([transforms.ToTensor(),
                                #transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # gray -> GRB 3 channel (lambda function)
                                #transforms.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0])
                               ])  # for grayscale images

# tiles dataset
tiles_dataset = Dataset(dir_path='../WSI/tiles/', transform=transform, dataset='tr')

# Data loader 
train_loader = torch.utils.data.DataLoader(dataset=tiles_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
#valid_loader = torch.utils.data.DataLoader(dataset=tiles_dataset, batch_size=batch_size, shuffle=False)

# Create model
autoencoder = autoencoder(fc_hidden1=fc_hidden1, drop_p=dropout_p, embed_dim=embed_dim).to(device)
    
# Recover model                                
if start_epoch != 0:
    model_load_path = os.path.join(save_model_path, 'model_epoch{}.pth'.format(start_epoch))
    autoencoder.load_state_dict(torch.load(model_load_path))

print("Using", torch.cuda.device_count(), "GPU")
model_params = list(autoencoder.parameters())
optimizer = torch.optim.Adam(model_params, lr=learning_rate)
# Recover optimizer                                
if start_epoch != 0:
    optimizer_load_path = os.path.join(save_model_path, 'optimizer_epoch{}.pth'.format(start_epoch))
    optimizer.load_state_dict(torch.load(optimizer_load_path))
    

epoch_train_losses = []
epoch_test_losses = []

# start training
for epoch in range(start_epoch, epochs):

    # train, test model
    X_train, z_train, train_losses = train(log_interval, autoencoder, device, train_loader, optimizer, epoch, save_model_path)
    #X_test, y_test, z_test, mu_test, logvar_test, epoch_test_loss = validation(resnet_vae, device, optimizer, valid_loader)

    # save results
    epoch_train_losses.append(train_losses)
    #epoch_test_losses.append(epoch_test_loss)

    
    # save all train test results
    A = np.array(epoch_train_losses)
    #C = np.array(epoch_test_losses)
    
    np.save(os.path.join(save_model_path, 'autoencoder_training_loss.npy'), A)
    np.save(os.path.join(save_model_path, 'X_ae_train_epoch{}.npy'.format(epoch + 1)), X_train) #save last batch
    np.save(os.path.join(save_model_path, 'z_ae_train_epoch{}.npy'.format(epoch + 1)), z_train)

FileNotFoundError: [Errno 2] No such file or directory: './ae_models/ae/model_epoch14.pth'

See loss curve

In [6]:
ae_loss_train = np.load(save_model_path+'/autoencoder_training_loss.npy')
print(ae_loss_train.shape)
print(ae_loss_train)

(6, 166494)
[[39465.8984375  39385.44140625 39076.1796875  ... 41471.4609375
  37312.58984375 36979.515625  ]
 [40819.234375   37718.46875    37465.67578125 ... 39333.5
  39408.71875    36919.8515625 ]
 [39553.1015625  37407.57421875 34201.50390625 ... 36990.578125
  37928.82421875 41303.40625   ]
 [35861.78125    36098.984375   41587.96875    ... 36376.97265625
  38091.4296875  40813.22265625]
 [36460.19921875 39564.625      36749.99609375 ... 38107.80078125
  34141.4375     38893.02734375]
 [39234.7109375  37108.7578125  38017.7578125  ... 38693.31640625
  41565.1875     35545.98046875]]


Predict

In [9]:
recover_epoch = 20

# Detect devices
use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu")   # use CPU or GPU

# Data loading parameters
params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 4, 'pin_memory': True} if use_cuda else {}
transform = transforms.Compose([transforms.ToTensor(),
                               ])

# tiles dataset
tiles_dataset = Dataset(dir_path='../WSI/tiles/', transform=transform, datset='tr')

# Data loader 
valid_loader = torch.utils.data.DataLoader(dataset=tiles_dataset, batch_size=batch_size, shuffle=False)

# Create model
autoencoder = autoencoder(fc_hidden1=fc_hidden1, drop_p=dropout_p, embed_dim=embed_dim).to(device)
    
# Recover model                                
model_load_path = os.path.join(save_model_path, 'model_epoch{}.pth'.format(recover_epoch))
autoencoder.load_state_dict(torch.load(model_load_path))

print("Using", torch.cuda.device_count(), "GPU")
model_params = list(autoencoder.parameters())
optimizer = torch.optim.Adam(model_params, lr=learning_rate)

# Recover optimizer                                
optimizer_load_path = os.path.join(save_model_path, 'optimizer_epoch{}.pth'.format(recover_epoch))
optimizer.load_state_dict(torch.load(optimizer_load_path))
    

# Evaluate

X_test, z_test, X_reconst_test, epoch_test_loss = evaluate(log_interval, autoencoder, device, optimizer, valid_loader)

        
np.save(os.path.join(save_model_path, 'AE_test_loss.npy'), epoch_test_loss)
np.save(os.path.join(save_model_path, 'X_ae_test_epoch{}.npy'.format(recover_epoch)), X_test)
np.save(os.path.join(save_model_path, 'z_ae_test_epoch{}.npy'.format(recover_epoch)), z_test)
np.save(os.path.join(save_model_path, 'X_reconst_test_epoch{}.npy'.format(recover_epoch)), X_reconst_test)


Using 1 GPU
Eval: [8000/1331953 (1%)]	Loss: 33068.367188
Eval: [16000/1331953 (1%)]	Loss: 35239.742188
Eval: [24000/1331953 (2%)]	Loss: 35465.242188
Eval: [32000/1331953 (2%)]	Loss: 34728.171875
Eval: [40000/1331953 (3%)]	Loss: 35035.937500
Eval: [48000/1331953 (4%)]	Loss: 32431.562500
Eval: [56000/1331953 (4%)]	Loss: 39313.914062
Eval: [64000/1331953 (5%)]	Loss: 38362.992188
Eval: [72000/1331953 (5%)]	Loss: 35454.273438
Eval: [80000/1331953 (6%)]	Loss: 50810.789062
Eval: [88000/1331953 (7%)]	Loss: 54876.113281
Eval: [96000/1331953 (7%)]	Loss: 37796.687500
Eval: [104000/1331953 (8%)]	Loss: 46583.910156
Eval: [112000/1331953 (8%)]	Loss: 52435.742188
Eval: [120000/1331953 (9%)]	Loss: 31942.667969
Eval: [128000/1331953 (10%)]	Loss: 44556.589844
Eval: [136000/1331953 (10%)]	Loss: 39604.101562
Eval: [144000/1331953 (11%)]	Loss: 41066.601562
Eval: [152000/1331953 (11%)]	Loss: 34181.765625
Eval: [160000/1331953 (12%)]	Loss: 34731.656250
Eval: [168000/1331953 (13%)]	Loss: 40257.625000
Eval: [1

# Internal Test set

In [9]:
recover_epoch = 20

# Detect devices
use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu")   # use CPU or GPU

# Data loading parameters
params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 4, 'pin_memory': True} if use_cuda else {}
transform = transforms.Compose([transforms.ToTensor(),
                               ])

# tiles dataset
tiles_dataset = Dataset(dir_path='../WSI/tiles/', transform=transform, dataset='ts')

# Data loader 
valid_loader = torch.utils.data.DataLoader(dataset=tiles_dataset, batch_size=batch_size, shuffle=False)

# Create model
autoencoder = autoencoder(fc_hidden1=fc_hidden1, drop_p=dropout_p, embed_dim=embed_dim).to(device)
    
# Recover model                                
model_load_path = os.path.join(save_model_path, 'model_epoch{}.pth'.format(recover_epoch))
autoencoder.load_state_dict(torch.load(model_load_path))

print("Using", torch.cuda.device_count(), "GPU")
model_params = list(autoencoder.parameters())
optimizer = torch.optim.Adam(model_params, lr=learning_rate)

# Recover optimizer                                
optimizer_load_path = os.path.join(save_model_path, 'optimizer_epoch{}.pth'.format(recover_epoch))
optimizer.load_state_dict(torch.load(optimizer_load_path))
    

# Evaluate

X_test, z_test, X_reconst_test, epoch_test_loss = evaluate(log_interval, autoencoder, device, optimizer, valid_loader)

    
        
np.save(os.path.join(save_model_path, 'AE_internal_test_loss.npy'), epoch_test_loss)
np.save(os.path.join(save_model_path, 'X_ae_internal_test_epoch{}.npy'.format(recover_epoch)), X_test)
np.save(os.path.join(save_model_path, 'z_ae_internal_test_epoch{}.npy'.format(recover_epoch)), z_test)
np.save(os.path.join(save_model_path, 'X_reconst_internal_test_epoch{}.npy'.format(recover_epoch)), X_reconst_test)

Using 1 GPU
Eval: [8000/207736 (4%)]	Loss: 35052.511719
Eval: [16000/207736 (8%)]	Loss: 43780.007812
Eval: [24000/207736 (12%)]	Loss: 37405.625000
Eval: [32000/207736 (15%)]	Loss: 36209.277344
Eval: [40000/207736 (19%)]	Loss: 38906.773438
Eval: [48000/207736 (23%)]	Loss: 35476.375000
Eval: [56000/207736 (27%)]	Loss: 37232.390625
Eval: [64000/207736 (31%)]	Loss: 58670.800781
Eval: [72000/207736 (35%)]	Loss: 34106.660156
Eval: [80000/207736 (39%)]	Loss: 33266.226562
Eval: [88000/207736 (42%)]	Loss: 34276.140625
Eval: [96000/207736 (46%)]	Loss: 33910.109375
Eval: [104000/207736 (50%)]	Loss: 37844.257812
Eval: [112000/207736 (54%)]	Loss: 32579.839844
Eval: [120000/207736 (58%)]	Loss: 32866.039062
Eval: [128000/207736 (62%)]	Loss: 34668.191406
Eval: [136000/207736 (65%)]	Loss: 32415.375000
Eval: [144000/207736 (69%)]	Loss: 40883.664062
Eval: [152000/207736 (73%)]	Loss: 30642.175781
Eval: [160000/207736 (77%)]	Loss: 31429.900391
Eval: [168000/207736 (81%)]	Loss: 35718.027344
Eval: [176000/20

MemoryError: Unable to allocate 9.51 GiB for an array with shape (207736, 3, 64, 64) and data type float32

# External Test Set

In [3]:
recover_epoch = 20

# Detect devices
use_cuda = torch.cuda.is_available()                   # check if GPU exists
device = torch.device("cuda" if use_cuda else "cpu")   # use CPU or GPU

# Data loading parameters
params = {'batch_size': batch_size, 'shuffle': True, 'num_workers': 4, 'pin_memory': True} if use_cuda else {}
transform = transforms.Compose([transforms.ToTensor(),
                                # Normalize to have tr dataset mean and std (params calculated in norm_params.ipynb
                                #transforms.Normalize(mean=[-0.2792, -0.2173, -0.0899], std=[1.2762, 1.2154, 1.0878], inplace=True)
                               ])

# tiles dataset
tiles_dataset = Dataset(dir_path='../WSI/tiles/', transform=transform, dataset='ext')

# Data loader 
valid_loader = torch.utils.data.DataLoader(dataset=tiles_dataset, batch_size=batch_size, shuffle=False)

# Create model
autoencoder = autoencoder(fc_hidden1=fc_hidden1, drop_p=dropout_p, embed_dim=embed_dim).to(device)
    
# Recover model                                
model_load_path = os.path.join(save_model_path, 'model_epoch{}.pth'.format(recover_epoch))
autoencoder.load_state_dict(torch.load(model_load_path))

print("Using", torch.cuda.device_count(), "GPU")
model_params = list(autoencoder.parameters())
optimizer = torch.optim.Adam(model_params, lr=learning_rate)

# Recover optimizer                                
optimizer_load_path = os.path.join(save_model_path, 'optimizer_epoch{}.pth'.format(recover_epoch))
optimizer.load_state_dict(torch.load(optimizer_load_path))
    

# Evaluate
X_test, z_test, X_reconst_test, epoch_test_loss = evaluate(log_interval, autoencoder, device, optimizer, valid_loader)

        
np.save(os.path.join(save_model_path, 'AE_ext_test_loss.npy'), epoch_test_loss)
np.save(os.path.join(save_model_path, 'X_ae_ext_test_epoch{}.npy'.format(recover_epoch)), X_test)
np.save(os.path.join(save_model_path, 'z_ae_ext_test_epoch{}.npy'.format(recover_epoch)), z_test)
#np.save(os.path.join(save_model_path, 'X_reconst_ext_test_epoch{}.npy'.format(recover_epoch)), X_reconst_test)

Using 1 GPU
Eval: [8000/584729 (1%)]	Loss: 37477.195312
Eval: [16000/584729 (3%)]	Loss: 38827.632812
Eval: [24000/584729 (4%)]	Loss: 36108.937500
Eval: [32000/584729 (5%)]	Loss: 35325.718750
Eval: [40000/584729 (7%)]	Loss: 53975.320312
Eval: [48000/584729 (8%)]	Loss: 69858.546875
Eval: [56000/584729 (10%)]	Loss: 39068.500000
Eval: [64000/584729 (11%)]	Loss: 37333.078125
Eval: [72000/584729 (12%)]	Loss: 39311.226562
Eval: [80000/584729 (14%)]	Loss: 44708.375000
Eval: [88000/584729 (15%)]	Loss: 42726.285156
Eval: [96000/584729 (16%)]	Loss: 36850.140625
Eval: [104000/584729 (18%)]	Loss: 41644.101562
Eval: [112000/584729 (19%)]	Loss: 38705.960938
Eval: [120000/584729 (21%)]	Loss: 37279.312500
Eval: [128000/584729 (22%)]	Loss: 51444.152344
Eval: [136000/584729 (23%)]	Loss: 42443.140625
Eval: [144000/584729 (25%)]	Loss: 42550.613281
Eval: [152000/584729 (26%)]	Loss: 43150.886719
Eval: [160000/584729 (27%)]	Loss: 42368.843750
Eval: [168000/584729 (29%)]	Loss: 45217.488281
Eval: [176000/584729